# Recurrent neural networks
[Recurrent neural networks](https://en.wikipedia.org/wiki/Recurrent_neural_network) are used for sequence data, that is data in which each item depends on one or more of the previous items. Examples of this type of data are time series and text. We can look at a simple RNN with one single hidden layer to understand how it works. The input data at time t is sent to the output layer through the hidden layer and also back to the hidden layer to be used in combination with the next input. The loop works like a memory and allows the network to learn the dependency between elements in the sequence. 

![RNN - Wikipedia, By fdeloche - Own work, CC BY-SA 4.0](images/recurrent_neural_network.svg)
(Image from Wikipedia. Credit: By fdeloche - Own work, CC BY-SA 4.0)

In [2]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import warnings
import torch
import torch.nn as nn
import torchvision
warnings.filterwarnings('ignore')
print("NumPy version: %s"%np.__version__)
print("Pandas version: %s"%pd.__version__)
print("PyTorch version: %s"%torch.__version__)

NumPy version: 1.23.1
Pandas version: 1.4.3
PyTorch version: 1.13.0


We can compute the preactivation of the hidden layer by two matrix multiplications, one to weight the input data and another to weight the result of the previous input data.

$$z_h^{(t)} = W_{xh}x^{(t)} + W_{hh}h^{(t-1)} + b_h$$

The output of the hidden layer is then computed by applying an activation function $\sigma_h$ to the result of the preactivation

$$h^{(t)} = \sigma_h (W_{xh}x^{(t)} + W_{hh}h^{(t-1)} + b_h)$$

Now we implement a small RNN with one hidden layer. Let's say the input data is an array of size 5 so that the input size of the hidden layer is 5. We set the size of the output of the hidden layer to be an array of size 2. With these settings it's easy to compute the shape of the $W_{xh}$ matrix to be 2x5 and the shape of the $W_{hh}$ matrix to be 2x2. The [RNN](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html) implementation in PyTorch builds the matrices from the same parameters. 

In [3]:
torch.manual_seed(1)

rnn_layer = nn.RNN(input_size=5, hidden_size=2, num_layers=1, batch_first=True) 

w_xh = rnn_layer.weight_ih_l0
w_hh = rnn_layer.weight_hh_l0
b_xh = rnn_layer.bias_ih_l0
b_hh = rnn_layer.bias_hh_l0

print('W_xh shape:', w_xh.shape)
print('W_hh shape:', w_hh.shape)
print('b_xh shape:', b_xh.shape)
print('b_hh shape:', b_hh.shape)

W_xh shape: torch.Size([2, 5])
W_hh shape: torch.Size([2, 2])
b_xh shape: torch.Size([2])
b_hh shape: torch.Size([2])


We can compute the output of the RNN instance for a sequence of three inputs and compare the result with that computed using the formula we have described above. 

In [5]:
x_seq = torch.tensor([[1.0]*5, [2.0]*5, [3.0]*5]).float()
x_seq

tensor([[1., 1., 1., 1., 1.],
        [2., 2., 2., 2., 2.],
        [3., 3., 3., 3., 3.]])

In [4]:
## output of the simple RNN:
output, hn = rnn_layer(torch.reshape(x_seq, (1, 3, 5)))

## manually computing the output:
out_man = []
for t in range(3):
    xt = torch.reshape(x_seq[t], (1, 5))
    print(f'Time step {t} =>')
    print('   Input           :', xt.numpy())
    
    ht = torch.matmul(xt, torch.transpose(w_xh, 0, 1)) + b_xh    
    print('   Hidden          :', ht.detach().numpy())
    
    if t>0:
        prev_h = out_man[t-1]
    else:
        prev_h = torch.zeros((ht.shape))

    ot = ht + torch.matmul(prev_h, torch.transpose(w_hh, 0, 1)) + b_hh
    ot = torch.tanh(ot)
    out_man.append(ot)
    print('   Output (manual) :', ot.detach().numpy())
    print('   RNN output      :', output[:, t].detach().numpy())
    print()

Time step 0 =>
   Input           : [[1. 1. 1. 1. 1.]]
   Hidden          : [[-0.4701929  0.5863904]]
   Output (manual) : [[-0.3519801   0.52525216]]
   RNN output      : [[-0.3519801   0.52525216]]

Time step 1 =>
   Input           : [[2. 2. 2. 2. 2.]]
   Hidden          : [[-0.88883156  1.2364397 ]]
   Output (manual) : [[-0.68424344  0.76074266]]
   RNN output      : [[-0.68424344  0.76074266]]

Time step 2 =>
   Input           : [[3. 3. 3. 3. 3.]]
   Hidden          : [[-1.3074701  1.886489 ]]
   Output (manual) : [[-0.8649416   0.90466356]]
   RNN output      : [[-0.8649416   0.90466356]]



Like the other neural networks that we have seen so far, the weights in a RNN are learnt through backpropagation. The loop introduced in a RNN with many layers may result in one of two opposite problems: exploding gradients or vanishing gradients. The two outcomes depend on the value of the $W_{hh}$ matrix that are computed multiple times depending on the lenght of the sequence we consider to be relevant for the output. If the $|W_{hh}| > 1$ we may face the problem of exploding gradients, on the contrary if $|W_{hh}| < 1$ we may face the problem of vanishing gradients. These problems can be addressed by limiting the length of the sequence we want to take into account for the output. Another apporach is to use the Long Short-Term Memory cells. 